In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn.metrics import confusion_matrix,roc_auc_score,roc_curve
from sklearn.ensemble import GradientBoostingClassifier

C:\Users\masato\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [12]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [13]:
train_data.loc[train_data["Gender"] == "Male", "Gender"] = 0
train_data.loc[train_data["Gender"] == "Female", "Gender"] = 1
X = train_data.loc[:,"Age":"AG_ratio"]
y = train_data.disease

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X)
X_std = sc.transform(X)

In [15]:
x_train,x_test,y_train,y_test = train_test_split(X_std,y,stratify=y,random_state=0)

In [16]:
gbrt = GradientBoostingClassifier(random_state=0)

In [17]:
parameter = {"n_estimators": np.arange(60,200,20),
             "learning_rate": np.arange(0.01,0.2,0.01),
             "max_depth": np.arange(2,5,1)}

In [26]:
grid = GridSearchCV(estimator=gbrt,param_grid=parameter,cv=5,n_jobs=7,scoring="f1")

In [27]:
grid.fit(x_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=7,
       param_grid={'n_estimators': array([ 60,  80, 100, 120, 140, 160, 180]), 'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19]), 'max_depth': array([2, 3, 4])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [24]:
grid.best_params_

{'learning_rate': 0.17, 'max_depth': 4, 'n_estimators': 60}

In [28]:
y_pred_proba = grid.predict_proba(x_test)[:,1]
auc_score = roc_auc_score(y_true=y_test,y_score=y_pred_proba)
print(auc_score)

0.9379125780553078


In [ ]:
test_data["Gender"] = test_data["Gender"].apply(lambda x: 1 if x=="Male" else 0)
test_pred_prob = forest.predict_proba(test_data)[:,1]
submit = pd.Series(data=test_pred_prob)
submit.to_csv("forest_proba.csv")